# Title
[]()

In [19]:
import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
import time
import re
from itertools import product
import openai

from response_processing import *
from article_processing import create_text_dict_from_folder
import traceback
from file_functions import *
from summary_chain import *

In [20]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set up

In [21]:
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()
chain_results_dict = dict()
qna_dict = dict()
main_text = dict()
text_dict = dict()

# Load previously scraped articles

In [22]:
scraping_folder_path = '../web_articles' # ** UPDATE REQUIRED**
text_id = 0

main_text[text_id] = loadpickle('text_dict_from_web2023-06-15_1057.sav', scraping_folder_path)
for key in main_text[text_id]:
    print(f"{key}: ", main_text[text_id][key].split('\n\n')[0])


Time completed: 2023-06-18 15:16:58.734743
Dictionary keys: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0:  Endovascular thrombectomy with or without thrombolysis bridging in patients with acute ischaemic stroke: protocol for a systematic review, meta-analysis of randomised trials and cost-effectiveness analysis
1:  Trend of risk and correlates of under-five child undernutrition in Bangladesh: an analysis based on Bangladesh Demographic and Health Survey data, 2007–2017/2018
2:  Analysing COVID-19 treatment outcomes in dedicated wards at a large university hospital in northern Poland: a result-based observational study
3:  SARS-CoV-2 cross-sectional seroprevalence study among public school staff in Metro Vancouver after the first Omicron wave in British Columbia, Canada
4:  Validation of diagnosis of acute myocardial infarction and stroke in electronic medical records: a primary care cross-sectional study in Madrid, Spain (the e-MADVEVA Study)
5:  Clinical characteristics of carbapenem-resistant Kl

In [23]:
# Pick text #7
text_dict[1] = main_text[text_id][7]
text_dict[1]

'Conducting a systematic review and evaluation of commercially available mobile applications (apps) on a health-related topic: the TECH approach and a step-by-step methodological guide\n\n<div class="panel-panel panel-col">\n    <div><div class="panel-pane pane-highwire-markup author-affiliates col-narrow-12 author-affiliates-corresp article">\n  \n      \n  \n  <div class="pane-content">\n    <div class="highwire-markup"><div xmlns="http://www.w3.org/1999/xhtml" class="content-block-markup" xmlns:xhtml="http://www.w3.org/1999/xhtml"><div xmlns:xhtml="http://www.w3.org/1999/xhtml" class="contributors"><ol class="contributor-list" id="contrib-group-1"><li class="contributor" id="contrib-1"><a href="http://orcid.org/0000-0001-7801-7417" class="bmjj-markup-orcid-logo" target="_blank">http://orcid.org/0000-0001-7801-7417</a><span class="name">Norina Gasteiger</span><a id="xref-aff-1-1" class="xref-aff" href="#aff-1">1</a><span class="xref-sep">,</span><a id="xref-aff-2-1" class="xref-aff" 

In [24]:
text_dict.keys()

dict_keys([1])

# Trim text

In [7]:
import re
from IPython import display
def trim_text(text, regex='(.*)'):
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction)?.*(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict
    
# regex='.*<h2>Abstract</h2>.*(?:Introduction)?.*(<h2.*?>Introduction</h2>.*References)<.*'
processed, html_display = trim_text(text_dict[1])
html_display

## trimmed iteration 1

In [9]:
import re
from IPython import display
def trim_text(text, regex='(.*)'):
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction)?.*(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict
    
regex='.*<h2>Abstract</h2>.*(?:Introduction)?.*(<h2.*?>Introduction</h2>.*References)<.*' # Doesn't work becauses abstract has no "introduction"
regex = '(.*)'
regex='.*<h2>Abstract</h2>.*(<h2.*?>Introduction</h2>.*References)<.*'
processed, html_display = trim_text(text_dict[1], regex=regex)
html_display

## ** Final regex for trim_text

In [25]:
import re
from IPython import display
def trim_text(text, regex=None):
    if regex==None:
        regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
    try:
        processed = re.search(regex, text, re.DOTALL).group(1)
        html_display = display.HTML(processed)
    except: 
        print('Unable to parse article text')
        processed = '<Error parsing article text>' 
        html_display = processed
    return processed, html_display

def text_dict_from_web(article_dict, header=2, to_display=0,
        regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    regex_str = regex_str.replace('\d', f'{header}')
    regex = rf'{regex_str}'
    print(f'Regex pattern: {regex}')
    text_dict = dict()
    display_dict = dict()
    if type(to_display) != list:
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], regex)
        text_dict[article_key] = f"{article_dict[article_key]['title']}\n\n{trimmed_text}"
        if article_key in to_display:
            display_dict[article_key] = display
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict
    
regex='.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' # Doesn't work becauses abstract has no "introduction"


processed, html_display = trim_text(text_dict[1], regex=None)
html_display

# Prepare text_dict

In [26]:

processed

'<h2 class="">Introduction</h2><p id="p-11">With the rise in the use of smartphones and other mobile technologies, there has been an increase in the availability of health applications (mHealth apps) designed to be used by individuals for various health issues. Health apps can also support health and care professionals in their daily clinical practice by providing decision support, access to clinical guidelines and education and training.<a id="xref-ref-1-1" class="xref-bibr" href="#ref-1">1</a> In 2018, over 325\u2009000 health apps were developed,<a id="xref-ref-2-1" class="xref-bibr" href="#ref-2">2</a> covering many health conditions and targeted behaviours. For example, mHealth apps can help to support self-management of conditions like diabetes,<a id="xref-ref-3-1" class="xref-bibr" href="#ref-3">3</a> facilitate remote monitoring of patients with chronic conditions<a id="xref-ref-4-1" class="xref-bibr" href="#ref-4">4</a> or support patients with general behaviour change such as

In [28]:
text_dict[1] = text_dict[1].split('\n\n')[0] + '\n\n' + processed
text_dict[1]

'Conducting a systematic review and evaluation of commercially available mobile applications (apps) on a health-related topic: the TECH approach and a step-by-step methodological guide\n\n<h2 class="">Introduction</h2><p id="p-11">With the rise in the use of smartphones and other mobile technologies, there has been an increase in the availability of health applications (mHealth apps) designed to be used by individuals for various health issues. Health apps can also support health and care professionals in their daily clinical practice by providing decision support, access to clinical guidelines and education and training.<a id="xref-ref-1-1" class="xref-bibr" href="#ref-1">1</a> In 2018, over 325\u2009000 health apps were developed,<a id="xref-ref-2-1" class="xref-bibr" href="#ref-2">2</a> covering many health conditions and targeted behaviours. For example, mHealth apps can help to support self-management of conditions like diabetes,<a id="xref-ref-3-1" class="xref-bibr" href="#ref-3"

# Prompts

In [29]:
prep_step = [
    "1. Identify 1 or 2 key messages from this text that would make interesting or helpful health content. \
    \n2. Identify the methods and research findings that support the key message(s). \
    Include the key statistics that describe these research findings. \
    \n3. If applicable, get a brief description of the research participants, \
    such as age, sex, and health conditions. Otherwise, you can skip this step.\
    \n4. Identify why the research might be relevant to the public.\
    Exclude details that do not add value to the audience.\
    \n5. Use the information from the previous steps to",
]

summarize_task = [
    "summarize the research in a text message to someone you know.",
    # "summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]
edit_task = [
    "\n6. Evaluate whether or not your writing may be confusing or redundant. \
    \nIf so, re-write it so it is clear and concise. Otherwise, keep it the same. \
    \n7. Create an intriguing subject line for the message.\
    \n8. Your final message should be in this format:\
    \n<headline>\n\n<summary>.\
    \nDo not label the headline and summary.",
]

system_role = "You are a science writer texting friends, family, and colleagues about the science research you have read."
# system_role = "You are a journalist writing content based on science research articles."
prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
    columns=['prep_step', 'summarize_task', 'edit_task'])

user_simplify_task = [
    """If needed, rewrite the headline and text using terms appropriate for the audience. If not keep it the same.\
    Follow these steps to accomplish this: \
    \n1. Check if the content and language are appropriate for the audience. \
    \n2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n3. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n4. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience is""",
]

simplify_audience = [
    # "a lay audience",
    "people who are not science experts",
]

user_relevance_task = [
    """Rewrite the headline and text to include a statement of how it is relevant for the audience. \
    Follow these steps to accomplish this: \
    \n1. Think about why this might be relevant for the audience in the grand scheme of things.\
    \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, \
    add a sentence to inform the audience. Otherwise, keep it the same. \
    Avoid making claims that are not supported by science.\
    \n3. Modify the content if needed to reduce redundancy. \
    \n4. Check if the content and language are appropriate for the audience. \
    If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n5. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n6. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience consists of""",
]

relevance_audience = [
    "seniors",
    # "people who enjoy sports",
    # "people new to resistance training",
    "people starting an exercise program"
]

# Run

In [30]:
# Set parameters
iteration_id = 1
n_choices = 5
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = True
save = True
# save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'
folder_path = '../web_articles/2023-06-15/'

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, model=model, max_tokens=1000,
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=save_outputs
    )
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )

# # Create simple summaries
# audience = simplify_audience
# simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
#     chatbot_dict[chatbot_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Add relevance
# relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
#     chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Merge the results
# try:
#     qna_dict = merge_all_chaining_results(
#         chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#         empty_columns=empty_columns, chatbot_id=chatbot_id,
#         save_df=save, save_chatbot=save, 
#             csv_path=folder_path,
#     )
#     print(f'\nCompleted merge_all_chaining_results!:)')
# except Exception as error:
#     exc_type, exc_obj, tb = sys.exc_info()
#     f = tb.tb_frame
#     lineno = tb.tb_lineno
#     file = f.f_code.co_filename
#     print(f'An error occurred on line {lineno} in {file}: {error}')
#     print('Unable to merge results')
#     if save:
#         save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
#         print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]

**Text #1 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 5 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
text1_prompt00
	text
	folder
	system_role
	temperature
	max_tokens
	model
	qna
	summaries_dict
	article_title
	response_regex
	simple_summary_dict
	relevance_dict
	n_previous_prompts
Object saved as JSON: ../web_articles/2023-06-15///batch_Chaining_attributes_initial_2023-06-18_1519.json
Processing text1_prompt00...
Original summaries DataFrame shape: (5, 12)
	Original summaries Dataframe columns: Index(['choice', 'date', 'folder', 'article_title', 'system_role', 'model',
       'text', 'prep step', 'summarization task', 'edit task',
       'full summarization task', 'summary'],
      dtype='object')
File saved:  batch_Chaining_summaries_initial2023-06-18_1519.csv
Time completed: 2023-06-18 15:19:

,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary
0,2023-06-18 1519,,Conducting a systematic review and evaluation of commercially available mobile applications (app...,1,"You are a science writer texting friends, family, and colleagues about the science research you ...",gpt-3.5-turbo-16k-0613,Conducting a systematic review and evaluation of commercially available mobile applications (app...,1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,summarize the research in a text message to someone you know.,"6. Evaluate whether or not your writing may be confusing or redundant. \nIf so, re-write it ...",1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,Conducting a systematic review and evaluation of commercially available mobile applications (app...
1,2023-06-18 1519,,Conducting a systematic review and evaluation of commercially available mobile applications (app...,2,"You are a science writer texting friends, family, and colleagues about the science research you ...",gpt-3.5-turbo-16k-0613,Conducting a systematic review and evaluation of commercially available mobile applications (app...,1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,summarize the research in a text message to someone you know.,"6. Evaluate whether or not your writing may be confusing or redundant. \nIf so, re-write it ...",1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,Subject: New Research on Evaluating Health Apps\n\nHey there! I just read a fascinating research...
2,2023-06-18 1519,,Conducting a systematic review and evaluation of commercially available mobile applications (app...,3,"You are a science writer texting friends, family, and colleagues about the science research you ...",gpt-3.5-turbo-16k-0613,Conducting a systematic review and evaluation of commercially available mobile applications (app...,1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,summarize the research in a text message to someone you know.,"6. Evaluate whether or not your writing may be confusing or redundant. \nIf so, re-write it ...",1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,Scientific Research: Conducting Systematic Reviews of Health Apps\n\nIntroduction:\nThe use of h...
3,2023-06-18 1519,,Conducting a systematic review and evaluation of commercially available mobile applications (app...,4,"You are a science writer texting friends, family, and colleagues about the science research you ...",gpt-3.5-turbo-16k-0613,Conducting a systematic review and evaluation of commercially available mobile applications (app...,1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,summarize the research in a text message to someone you know.,"6. Evaluate whether or not your writing may be confusing or redundant. \nIf so, re-write it ...",1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,Conducting a systematic review and evaluation of commercially available mobile applications (app...
4,2023-06-18 1519,,Conducting a systematic review and evaluation of commercially available mobile applications (app...,5,"You are a science writer texting friends, family, and colleagues about the science research you ...",gpt-3.5-turbo-16k-0613,Conducting a systematic review and evaluation of commercially available mobile applications (app...,1. Identify 1 or 2 key messages from this text that would make interesting or helpful health con...,summarize the research in a text message to someone you know.,"6. Evaluate whether or not your writing may be confusing or redundant. \nIf so, re-write it ...",1. Ide

# *End of Page*